In [1]:
import pprint
from sklearn.cluster import KMeans

def clustering(num_clusters, data,data_labels):
    mu = {}
    sigma = {}
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(data)
    l = [data_labels[np.where(kmeans.labels_ == i)] for i in range(num_clusters\
)]

    #pprint.pprint(l)
    sigma_0 = {i: np.std(data[np.where(kmeans.labels_ == i)], axis=0) for i in
               range(num_clusters)}
    for l in range(0, num_clusters):
        mu['cluster_' + str(l)] = kmeans.cluster_centers_[
            l] 
    for l in range(0, num_clusters):
        sigma['cluster_' + str(l)] = sigma_0[l] 

    return mu, sigma

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets, layers, models
#from networks.dense_net import DenseNet
from six.moves import xrange
import pprint
from scipy import special
import tensorflow_probability as tfp


def Train_and_Test_RBF(X_train,X_test,y_train,y_test,cancer_size,minority_idx):
            

       
        normal_size = len(X_train) - cancer_size
        print('len(data_samples) ',len(X_train))
        print('cancer_size ' , cancer_size)
        print('normal_size ' , normal_size)
 
        lables  = [] # np.zeros([replication, rep_length]) 
       
        # this method will define the network and train the model
        batch_size = 25 # X_test.shape[0]
        z_dim = 500
        tfd = tfp.distributions
        K = 5
        runs = 300
        mu, sigma = clustering(K,X_train,y_train)
        

        encoder = tf.keras.Sequential([
                    layers.Dense(1000, activation=tf.nn.sigmoid,name="e_1")
                    
            

                ])

        decoder = {}
        for k in range(K):
            key = 'cluster_'+str(k)
            decoder[key] =  tf.keras.Sequential([
                  
                  layers.Dense(1000, activation=tf.nn.sigmoid,name="d1")
                ])


        tf.compat.v1.disable_eager_execution()
        x = tf.compat.v1.placeholder(tf.float32, shape=[None, X_train.shape[1]])
        #encoded = encoder(x)

        mean = layers.Dense(z_dim, tf.nn.sigmoid,name="mean_latent")(x) #encoded
        sigma = layers.Dense(z_dim, tf.nn.sigmoid,name="var_latent")(x) #encoded

        z = mean + tf.multiply(tf.sqrt(tf.exp(sigma)),
                                       tf.random.normal(shape=(tf.shape(mean)[0], z_dim)))
        x_reco = {}
        mean_decoder = {}
        sigma_decoder = {}
        rec_loss_per_cluster = {}
        for k in range(K):
            key = 'cluster_'+str(k)
            x_reco[key] = decoder[key](z)
            mean_decoder[key] = layers.Dense(X_train.shape[1], tf.nn.sigmoid,name="mean_decoder")(z) #x_reco[key]
            sigma_decoder[key] = layers.Dense(X_train.shape[1], tf.nn.sigmoid,name="var_decoder")(z)#x_reco[key]
               
            rec_loss_per_cluster[key] = tf.reduce_sum(tf.square(mean_decoder[key] - mu[key]),axis=1)#+\
            #tf.reduce_sum(sigma_decoder[key][0] - sigma[key],axis=1)
       
        
        reconstruction_term =  tf.reduce_mean(list(rec_loss_per_cluster.values()) )

        mvn = tfd.MultivariateNormalDiag(loc=[0. for i in range(z_dim)],scale_diag=[1. for i in range(z_dim)])
        #kl_divergence = tf.reduce_sum(tf.keras.metrics.kullback_leibler_divergence(z,mvn.sample(z.shape[0])))#self.mu_d[key]/255., x_reco[key]), axis=[1,2])

        cost = tf.reduce_mean(reconstruction_term) #+ tf.reduce_mean(kl_divergence)
        optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
        

        n_minibatches = int(X_train.shape[0] / batch_size)
        if(int(X_train.shape[0] % batch_size)) >0:
            n_minibatches +=1
        print("Number of minibatches: ", n_minibatches)

        sess = tf.compat.v1.InteractiveSession()
        init = tf.compat.v1.global_variables_initializer()
        sess.run(init)
        for epoch in range(runs):
                    pbar = tf.keras.utils.Progbar(n_minibatches)
                    for i in range(n_minibatches):
                        x_batch = X_train[i * batch_size:(i + 1) * batch_size] 
                        cost_, _,z_print = sess.run((cost, optimizer, z), feed_dict={x: x_batch})
                        if epoch%100 == 0:
                            pbar.add(1, [("cost", cost_),( "epoch",epoch)])
                        
        train_z_test = mean + tf.multiply(tf.sqrt(tf.exp(sigma)),
                                       tf.random.normal(shape=(X_train.shape[0], z_dim)))
        test_z_test = mean + tf.multiply(tf.sqrt(tf.exp(sigma)),
                                       tf.random.normal(shape=(X_test.shape[0], z_dim)))
      
        z_train = sess.run(train_z_test, feed_dict={x: X_train})
        z_test =  sess.run(test_z_test, feed_dict={x: X_test})
        
        '''
        
        collection = []#np.zeros([data_samples+replication, rep_length])  
        lables  = [] 
        class_list=[]
        replication = cancer_size - normal_size
        total_number_of_replications = 0
        for i in range(len(X_train)):
            round = replication//normal_size
            samples_number=normal_size
            
            if y_train[i] == minority_idx:
            
                if (replication%normal_size) != 0:
                  round=round +1
                if(total_number_of_replications == replication):
                    round = 0
                #print(round)
                for idx1 in range(round): #cancer_size // normal_size + 1):
                        total_number_of_replications +=1
                        synth_train = sess.run(train_z_test, feed_dict={x: X_train[i].reshape(1,X_train.shape[1])})
                        
                        collection.append(synth_train[0] )#optimized_code[0])
                        lables.append(minority_idx)
        if len(collection) > 0:
            collection = np.vstack((z_train,collection))

            print(y_train.shape)
            print(np.array(lables).shape)

            lables = np.hstack((y_train,lables))
        else:
            collection =  z_train
            lables = y_train
        
        
        '''
        
        
        
        sess.close()
        return z_train,z_test,y_train
                        


In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets, layers, models
#from networks.dense_net import DenseNet
from six.moves import xrange
import pprint
from scipy import special
import tensorflow_probability as tfp


def Train_and_Test_VAE(X_train,X_test,y_train,y_test):
        # this method will define the network and train the model
        batch_size = 25 # X_test.shape[0]
        z_dim = 250
        tfd = tfp.distributions
        
        runs = 300
        

        encoder = tf.keras.Sequential([
                    layers.Dense(500, activation=tf.nn.sigmoid,name="e_1")
                    
                   

                ])

        
        decoder =  tf.keras.Sequential([
        
                  layers.Dense(500, activation=tf.nn.sigmoid,name="d_2")
                 
                ])


        tf.compat.v1.disable_eager_execution()
        x = tf.compat.v1.placeholder(tf.float32, shape=[None, X_train.shape[1]])
        encoded = encoder(x)

        mean = layers.Dense(z_dim, tf.nn.sigmoid,name="mean_latent")(x)#encoded)
        sigma = layers.Dense(z_dim, tf.nn.sigmoid,name="var_latent")(x)#encoded)

        z = mean + tf.multiply(tf.sqrt(tf.exp(sigma)),
                                       tf.random.normal(shape=(batch_size, z_dim)))
        
        x_reco = decoder(z)
        mean_decoder = layers.Dense(X_train.shape[1], tf.nn.sigmoid,name="mean_decoder")(z)#x_reco)
        sigma_decoder = layers.Dense(X_train.shape[1], tf.nn.sigmoid,name="var_decoder")(z)#x_reco)
               
        rec_loss_per_cluster = tf.reduce_sum(tf.square(mean_decoder- x),axis=1)#+\
            #tf.reduce_sum(sigma_decoder[key][0] - sigma[key],axis=1)
       
        
        reconstruction_term =  tf.reduce_mean(rec_loss_per_cluster )

        mvn = tfd.MultivariateNormalDiag(loc=[0. for i in range(z_dim)],scale_diag=[1. for i in range(z_dim)])
        kl_divergence = tf.reduce_sum(tf.keras.metrics.kullback_leibler_divergence(z,mvn.sample(z.shape[0])))#self.mu_d[key]/255., x_reco[key]), axis=[1,2])

        cost = tf.reduce_mean(reconstruction_term) + tf.reduce_mean(kl_divergence)
        optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)
        

        n_minibatches = int(X_train.shape[0] / batch_size)
        print("Number of minibatches: ", n_minibatches)

        sess = tf.compat.v1.InteractiveSession()
        init = tf.compat.v1.global_variables_initializer()
        sess.run(init)
        for epoch in range(runs):
                    pbar = tf.keras.utils.Progbar(n_minibatches)
                    for i in range(n_minibatches):
                        x_batch = X_train[i * batch_size:(i + 1) * batch_size] 
                        cost_, _,z_print = sess.run((cost, optimizer, z), feed_dict={x: x_batch})
                        pbar.add(1, [("cost", cost_),( "epoch",epoch)])
                        
        train_z_test = mean + tf.multiply(tf.sqrt(tf.exp(sigma)),
                                       tf.random.normal(shape=(X_train.shape[0], z_dim)))
        test_z_test = mean + tf.multiply(tf.sqrt(tf.exp(sigma)),
                                       tf.random.normal(shape=(X_test.shape[0], z_dim)))
        x_reco = {}
        z_train = sess.run(train_z_test, feed_dict={x: X_train})
        z_test =  sess.run(test_z_test, feed_dict={x: X_test})
        
        sess.close()
        return z_train,z_test
                        


In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets, layers, models
#from networks.dense_net import DenseNet
from six.moves import xrange
import pprint
from scipy import special
import tensorflow_probability as tfp


def Train_and_Test_GMVAE(X_train,X_test,y_train,y_test):
        # this method will define the network and train the model
        batch_size = 25 # X_test.shape[0]
        hidden_dim = 500
        z_dim = 250
        tfd = tfp.distributions
        K = 5
        w_dim  = 50
        
        runs =300
        tf.compat.v1.disable_eager_execution()
        y = tf.compat.v1.placeholder(tf.float32, shape=[None, X_train.shape[1]])
        
        ########################## ENCODER PART ###################################
        # The representtions X
        
        Qx_y= tf.keras.Sequential([
                    layers.Dense(hidden_dim, activation=tf.nn.sigmoid,name="Qy_x_1")

                ]) 
        Qx_y_mean = layers.Dense(z_dim, tf.nn.sigmoid,name="Qx_y_mean")(Qx_y(y))
        Qx_y_var= layers.Dense(z_dim, tf.nn.sigmoid,name="Qx_y_var")(Qx_y(y))

        Qx_y_samples= Qx_y_mean + tf.multiply(tf.sqrt(tf.exp(Qx_y_var)),
                                       tf.random.normal(shape=(batch_size, z_dim)))
        
        #................................................................................
        
        
        Qw_y =   tf.keras.Sequential([
                    layers.Dense(hidden_dim, activation=tf.nn.sigmoid,name="Qw_y_1")
                
                ])
        Qw_y_mean = layers.Dense(w_dim, tf.nn.sigmoid,name="Qw_y_mean")(Qw_y(y))
        Qw_y_var  = layers.Dense(w_dim, tf.nn.sigmoid,name="Qw_y_var")(Qw_y(y))
         
                                                    
        Qw_y_samples= Qw_y_mean + tf.multiply(tf.sqrt(tf.exp(Qw_y_var)),
                                       tf.random.normal(shape=(batch_size, w_dim)))    
            
        
        #...............................................................................
        # The components coeff Z
        xw = tf.concat([Qw_y_samples, Qx_y_samples],1, name='xw_concat')
        Qz_xw = tf.keras.Sequential([
                    layers.Dense(z_dim, activation=tf.nn.sigmoid,name="Qxw_z_1")
                ])
        Qz_xw_mean = layers.Dense(K, tf.nn.sigmoid,name="Qz_xw_mean")(Qz_xw(xw))
        Qz_xw_var= layers.Dense(K, tf.nn.sigmoid,name="Qz_xw_var")(Qz_xw(xw))

        Qz_samples= Qz_xw_mean + tf.multiply(tf.sqrt(tf.exp(Qz_xw_var)),
                                       tf.random.normal(shape=(batch_size, K)))
        
        
        ###################################
        ########################################
        ####################### Decoder ###########################################
        
        # Px_wz, https://arxiv.org/pdf/1611.02648.pdf
        #More specifically, the neural network parameterised by β outputs a set of K means µzk and K variances σ**2 zk
        #, given w as input. A one-hot vector z is sampled from the mixing probability π, which chooses one component 
        #from the Gaussian mixture. We set the parameter πk = K**−1 to make z uniformly
        #distributed. The generative and variational views of this model are depicte
        #......................Px_wz_mean .....................
        #wz = tf.concat([Qw_samples, Qz_samples],1, name='wz_concat')
        Px_wz_mean_dict = {}
        for k in range(K): 
            key = 'cluster_'+str(k)
            
            Px_wz_mean = layers.Dense(z_dim, tf.nn.sigmoid,name="Py_x_mean")(Qw_y_samples)
            Px_wz_mean_dict[key] = Px_wz_mean
        #........................... Px_wz_var ......................  
        Px_wz_var_dict = {}
        for k in range(K):                                    
            key = 'cluster_'+str(k)
            Px_wz_var = layers.Dense(z_dim, tf.nn.sigmoid,name="Py_x_mean")(Qw_y_samples)
            Px_wz_var_dict[key] = Px_wz_var
        
         #.................. sampling Px_wz ...........
        # Sample Z as one-hot categioral 
        temperature = 1e-5
        z_dist = tfd.RelaxedOneHotCategorical(temperature, probs=Qz_samples)
        z_samples = z_dist.sample(1)
                                            
        #......... Define the GMM to sample from 
        gm = tfd.MixtureSameFamily(
        mixture_distribution= tfd.Categorical(     
        probs=Qz_samples),
            components_distribution= tfd.MultivariateNormalDiag(
                loc=[[Px_wz_mean_dict['cluster_'+str(k)][i] for k in range(K)]for i in range(batch_size) ],
                scale_diag=[[tf.sqrt(tf.exp(Px_wz_var_dict['cluster_'+str(k)][i])) for k in range(K)]for i in range(batch_size) ],name="prior_code"))
        
        Px_wz_samples = gm.sample(1)
                                            
        #.......................Py_x ..............................
        Py_x =   tf.keras.Sequential([
                    layers.Dense(hidden_dim, activation=tf.nn.sigmoid,name="Qw_y_1")
                
                ])
        Py_x_mean = layers.Dense(X_train.shape[1], tf.nn.sigmoid,name="Py_x_mean")(Py_x(Px_wz_samples))
        Py_x_var  = layers.Dense(X_train.shape[1], tf.nn.sigmoid,name="Py_x_var")(Py_x(Px_wz_samples))
         
                                                    
        Py_x_samples= Py_x_mean + tf.multiply(tf.sqrt(tf.exp(Py_x_var)),
                                       tf.random.normal(shape=(batch_size, X_train.shape[1])))
        ################################################################################
       ####################################################### loss 
        
           # recons_loss, #log p(y|x) 
        #Py_x_dist =  tfp.distributions.MultivariateNormalDiag(
          #             loc=Py_x_mean, 
         #               scale_diag=Py_x_var,  name='Py_x') 
           
        #logy_x = Py_x_dist.log_prob(X_train)
        
        y_x_loss = tf.pow(tf.reduce_sum(tf.square(y-Py_x_samples),axis=1),.5)
           # q loss KL(q(x|y)||p(x|w, z))
        KLxy_xwz = 0.
        
        #for k in range(K):
         #      key = 'cluster_'+str(k)
        kl_divergence = tf.reduce_sum(tf.keras.metrics.kullback_leibler_divergence(Qx_y_samples,Px_wz_samples))
        '''
               KLxy_xwz += tf.reduce_sum((tf.math.log(Qx_y_var) - tf.math.log(Px_wz_var_dict[key]) -
            .5 * (1. - (Px_wz_var_dict[key]**2 + (Qx_y_mean - Px_wz_mean_dict[key] )**2) / Qx_y_var**2)),axis=1)
           '''
           #KL(q(w|y)||p(w))
        mvn = tfd.MultivariateNormalDiag(loc=[0. for i in range(w_dim)],scale_diag=[1. for i in range(w_dim)])
        KLwy_w = tf.reduce_sum(tf.keras.metrics.kullback_leibler_divergence(Qw_y_samples,mvn.sample(batch_size)))
                                              
        #KLwy_w =  tf.reduce_sum((tf.math.log(Qw_y_var) - tf.math.log(1.) -
         #   .5 * (1. - (Qw_y_var**2 + (Qw_y_mean - 0. )**2) / 1.**2)),axis=1)                                                                    
                                                                                 
           #KL(p(z|x, w)||p(z))
        mvn = tfd.MultivariateNormalDiag(loc=[0. for i in range(K)],scale_diag=[1. for i in range(K)])
        KLzxw_z = tf.reduce_sum(tf.keras.metrics.kullback_leibler_divergence(Qz_samples,mvn.sample(batch_size)))
                                                                                 
        #KLzxw_z = tf.reduce_sum((tf.math.log(Qz_xw_var) - tf.math.log(1.) -
         #   .5 * (1. - (Qz_xw_var**2 + (Qz_xw_mean - 0. )**2) / 1.**2)) ,axis=1)
        #tf.reduce_mean(logy_x)
                                                                          
        cost =- tf.reduce_mean(KLxy_xwz) - tf.reduce_mean(KLwy_w) - tf.reduce_mean(KLzxw_z)
        optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)
                                            
            
        n_minibatches = int(X_train.shape[0] / batch_size)
        print("Number of minibatches: ", n_minibatches)

        sess = tf.compat.v1.InteractiveSession()
        init = tf.compat.v1.global_variables_initializer()
        sess.run(init)
        for epoch in range(runs):
                    pbar = tf.keras.utils.Progbar(n_minibatches)
                    for i in range(n_minibatches):
                        x_batch = X_train[i * batch_size:(i + 1) * batch_size] 
                        cost_, _,z_print = sess.run((cost, optimizer, Qx_y_samples), feed_dict={y: x_batch})
                        pbar.add(1, [("cost", cost_),( "epoch",epoch)])
                        
        train_z_test = Qx_y_mean + tf.multiply(tf.sqrt(tf.exp(Qx_y_var)),
                                       tf.random.normal(shape=(X_train.shape[0], z_dim)))
        test_z_test = Qx_y_mean + tf.multiply(tf.sqrt(tf.exp(Qx_y_var)),
                                       tf.random.normal(shape=(X_test.shape[0], z_dim)))
        x_reco = {}
        z_train = sess.run(train_z_test, feed_dict={y: X_train})
        z_test =  sess.run(test_z_test, feed_dict={y: X_test})
        
        sess.close()
        return z_train,z_test       
        
                        


In [5]:
from sklearn.naive_bayes import GaussianNB
from sklearn import neighbors, datasets
from sklearn import tree
from sklearn import svm, datasets,gaussian_process, linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from scipy import interp
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score,roc_auc_score

def classifying_data(train_reps,test_reps,y_train,y_test):
     RF_classifier = RandomForestClassifier(n_estimators=100, criterion='gini',\
     random_state=0, max_depth=100,bootstrap=True)
     RF_classifier.fit(train_reps,y_train)
     y_score = RF_classifier.predict(test_reps)
     score_RF = accuracy_score(y_test, y_score, normalize=True,
                       sample_weight=None)
    
     SMO_classifier = svm.SVC( gamma='auto') #'scale'
     SMO_classifier.fit(train_reps,y_train)
     y_score = SMO_classifier.predict(test_reps)
     score_SMO = accuracy_score(y_test, y_score, normalize=True,
                       sample_weight=None)
    
     NB_classifier = GaussianNB(var_smoothing=100)
     NB_classifier.fit(train_reps,y_train)
     y_score = NB_classifier.predict(test_reps)
     score_NB = accuracy_score(y_test, y_score, normalize=True,
                       sample_weight=None)
    
     LR_classifier = linear_model.LogisticRegressionCV(multi_class="ovr",max_iter=500)
     LR_classifier.fit(train_reps,y_train)
     y_score = LR_classifier.predict(test_reps)
     LR_score = accuracy_score(y_test, y_score, normalize=True,
                       sample_weight=None)


     return score_RF,score_SMO,score_NB,LR_score

    

    

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn import preprocessing

data = pd.read_csv("data/SMK_CAN_187.csv")#,header=None)
print("data shape before split :", data.shape)

#...................................................................................... Normalization
data_normalized = preprocessing.scale(data.values[:,:-1])
y = data.values[:,-1] 
data_normalized = np.hstack((data_normalized,y[:,np.newaxis]))
print("data shape after normalization:", data_normalized.shape)


#.....................................................................................splitting into 10-folds
kf = KFold(n_splits=10,shuffle=True) # Define the split - into 10 folds 
kf.get_n_splits(data_normalized) # returns the number of splitting iterations in the cross-validator
#print(kf) 

#.....................................................................................start_training
RF_score_cumm,SMO_score_cumm,NB_score_cumm,LR_score_cumm = [],[],[],[]
VAE_RF_score_cumm,VAE_SMO_score_cumm,VAE_NB_score_cumm,VAE_LR_score_cumm = [],[],[],[]
GMV_RF_score_cumm,GMV_SMO_score_cumm,GMV_NB_score_cumm,GMV_LR_score_cumm = [],[],[],[]
RF_training_score = []


for train_index, test_index in kf.split(data_normalized):
     print("########################## fold #################")
     #print("TRAIN:", train_index, "TEST:", test_index)
     X_train, X_test = data_normalized[train_index,:-1], data_normalized[test_index,:-1]
     y_train, y_test = data_normalized[train_index,-1], data_normalized[test_index,-1]
     #print("TRAIN:", y_train, "TEST:", y_test)
     df = pd.DataFrame(y_train, columns=["val"])
     splits = df.groupby("val").val.apply(pd.Series.tolist).values

     majority_size = 0
     minority_idx = 0
     minority_size = 10000
     for i in range(splits.shape[0]):

                if len(splits[i]) > majority_size:
                    majority_size = len(splits[i])
                if len(splits[i]) < minority_size:
                    minority_size = len(splits[i])
                    minority_idx = splits[i][0]
     print("Majority Size : ",majority_size)
     
     # call the NN-model
     train_reps,test_reps,train_lables = Train_and_Test_RBF(X_train,X_test,y_train,y_test,majority_size,minority_idx)
     # classify the reps
     RF_score,SMO_score,NB_score,LR_score = classifying_data(train_reps,test_reps,train_lables,y_test ) #y_train
     #rf_train,SMO_train,NB_train,LR_train =  classifying_data(train_reps,train_reps,y_train,y_train )
     print("RF = ",RF_score, "SMO = ", SMO_score,"NB = ",NB_score,"LR = ",LR_score)
     RF_score_cumm.append(RF_score)
     SMO_score_cumm.append(SMO_score )
     NB_score_cumm.append(NB_score)
     LR_score_cumm.append(LR_score)
     #RF_training_score.append(NB_train)
     
     train_reps,test_reps = Train_and_Test_VAE(X_train,X_test,y_train,y_test)
     RF_score,SMO_score,NB_score,LR_score = classifying_data(train_reps,test_reps,y_train,y_test )
     print("RF = ",RF_score, "SMO = ", SMO_score,"NB = ",NB_score,"LR = ",LR_score)
     VAE_RF_score_cumm.append(RF_score)
     VAE_SMO_score_cumm.append(SMO_score) 
     VAE_NB_score_cumm.append(NB_score)
     VAE_LR_score_cumm.append(LR_score)
     
     
     train_reps,test_reps = Train_and_Test_GMVAE(X_train,X_test,y_train,y_test)
     RF_score,SMO_score,NB_score,LR_score = classifying_data(train_reps,test_reps,y_train,y_test )
     print("RF = ",RF_score, "SMO = ", SMO_score,"NB = ",NB_score,"LR = ",LR_score)
     GMV_RF_score_cumm.append(RF_score)
     GMV_SMO_score_cumm.append(SMO_score) 
     GMV_NB_score_cumm.append(NB_score)
     GMV_LR_score_cumm.append(LR_score)
     

print("_________________________Multi___________________________________")
print("$","{:.3f}".format(np.mean(RF_score_cumm)), " \pm  ",
                           "{:.3f}".format(np.std(RF_score_cumm)), "$ & ",
                           "$","{:.3f}".format(np.mean(SMO_score_cumm)), " \pm  ",
                           "{:.3f}".format(np.std(SMO_score_cumm)), "$ & ", 
                           "$","{:.3f}".format(np.mean(NB_score_cumm)), " \pm  ",
                           "{:.3f}".format(np.std(NB_score_cumm)), "$ & ",
                           "$","{:.3f}".format(np.mean(LR_score_cumm)), " \pm  ",
                           "{:.3f}".format(np.std(LR_score_cumm)), "$  ")
print("_________________________VAE___________________________________")
print("$","{:.3f}".format(np.mean(VAE_RF_score_cumm)), " \pm  ",
                           "{:.3f}".format(np.std(VAE_RF_score_cumm)), "$ & ",
                           "$","{:.3f}".format(np.mean(VAE_SMO_score_cumm)), " \pm  ",
                           "{:.3f}".format(np.std(VAE_SMO_score_cumm)), "$ & ", 
                           "$","{:.3f}".format(np.mean(VAE_NB_score_cumm)), " \pm  ",
                           "{:.3f}".format(np.std(VAE_NB_score_cumm)), "$ & ",
                           "$","{:.3f}".format(np.mean(VAE_LR_score_cumm)), " \pm  ",
                           "{:.3f}".format(np.std(VAE_LR_score_cumm)), "$")

print("_________________________GMVAE___________________________________")
print("$","{:.3f}".format(np.mean(GMV_RF_score_cumm)), " \pm  ",
                           "{:.3f}".format(np.std(GMV_RF_score_cumm)), "$ & ",
                           "$","{:.3f}".format(np.mean(GMV_SMO_score_cumm)), " \pm  ",
                           "{:.3f}".format(np.std(GMV_SMO_score_cumm)), "$ & ", 
                           "$","{:.3f}".format(np.mean(GMV_NB_score_cumm)), " \pm  ",
                           "{:.3f}".format(np.std(GMV_NB_score_cumm)), "$ & ",
                           "$","{:.3f}".format(np.mean(GMV_LR_score_cumm)), " \pm  ",
                           "{:.3f}".format(np.std(GMV_LR_score_cumm)), "$  ")





print("training score")
print(np.mean(RF_training_score), np.std(RF_training_score))

data shape before split : (186, 19994)
data shape after normalization: (186, 19994)
########################## fold #################
Majority Size :  88
len(data_samples)  167
cancer_size  88
normal_size  79
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
Number of minibatches:  7
7/7 [==============================] - 1s 194ms/step - cost: 7873.9316 - epoch: 200.0000
RF =  0.5263157894736842 SMO =  0.47368421052631576 NB =  0.5789473684210527 LR =  0.5789473684210527
Number of minibatches:  6
6/6 [==============================] - 0s 43ms/step - cost: 39388.2910 - epoch: 168.0000


6/6 [==============================] - 0s 43ms/step - cost: 38471.0228 - epoch: 299.0000
RF =  0.5263157894736842 SMO =  0.5263157894736842 NB =  0.5789473684210527 LR =  0.5789473684210527
Number of minibatches:  6
6/6 [==============================] - 1s 90ms/step - cost: -7595.7631 - epoch: 51.0000


6/6 [==============================] - 1s 91ms/step - cost: -7745.8879 - epoch: 235.0000


6/6 [==============================] - 1s 91ms/step - cost: -7905.5249 - epoch: 299.0000
RF =  0.5263157894736842 SMO =  0.47368421052631576 NB =  0.47368421052631576 LR =  0.5263157894736842
########################## fold #################
Majority Size :  89
len(data_samples)  167
cancer_size  89
normal_size  78
Number of minibatches:  7
7/7 [==============================] - 1s 193ms/step - cost: 5941.5170 - epoch: 200.0000
RF =  0.5789473684210527 SMO =  0.631578947368421 NB =  0.5789473684210527 LR =  0.5789473684210527
Number of minibatches:  6
6/6 [==============================] - 0s 43ms/step - cost: 39840.5378 - epoch: 113.0000


6/6 [==============================] - 0s 45ms/step - cost: 37858.7799 - epoch: 297.0000


...

6/6 [==============================] - 1s 91ms/step - cost: -7875.8790 - epoch: 196.0000


6/6 [==============================] - 1s 91ms/step - cost: -7744.1125 - epoch: 299.0000
RF =  0.42105263157894735 SMO =  0.2631578947368421 NB =  0.42105263157894735 LR =  0.42105263157894735
########################## fold #################
Majority Size :  86
len(data_samples)  167
cancer_size  86
normal_size  81
Number of minibatches:  7
7/7 [==============================] - 1s 195ms/step - cost: 7232.4263 - epoch: 200.0000
RF =  0.6842105263157895 SMO =  0.631578947368421 NB =  0.631578947368421 LR =  0.7894736842105263
Number of minibatches:  6
6/6 [==============================] - 0s 46ms/step - cost: 41019.1914 - epoch: 73.0000


6/6 [==============================] - 0s 46ms/step - cost: 38337.1735 - epoch: 257.0000


6/6 [==============================] - 0s 46ms/step - cost: 38092.0518 - epoch: 299.0000
RF =  0.6842105263157895 SMO =  0.631578947368421 NB =  0.42105263157894735 LR =  0.631578947368421
Number of minibatches:  6
6/6 [==============================] - 1s 93ms/step - cost: -7987.0183 - epoch: 140.0000


6/6 [==============================] - 1s 93ms/step - cost: -7665.7906 - epoch: 299.0000
RF =  0.47368421052631576 SMO =  0.631578947368421 NB =  0.5789473684210527 LR =  0.5789473684210527
########################## fold #################
Majority Size :  84
len(data_samples)  168
cancer_size  84
normal_size  84
Number of minibatches:  7
7/7 [==============================] - 1s 200ms/step - cost: 7660.5722 - epoch: 100.0000
